# Diamond Price Prediction using KNN & Streamlit Deployment

In [ ]:
1: End-to-End ML Pipeline
1 Load the dataset using pandas
2 Identify input features (X) and target variable (price)
3 Split the dataset into train and test (75:25 ratio)
4 Perform data preprocessing: categorical encoding and numerical scaling
5 Train a KNN Regressor model using sklearn
6 Evaluate the model using MAE, RMSE, and R2 score
7 Save the trained pipeline using pickle

# Load the Dataset

In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("diamonds.csv")
df.head(3)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31


# step - 1 : Understand the Data 
- Check columns, data types, target variable
- Identify numerical / categorical / text / image data

In [4]:
df.shape

(53940, 10)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


In [6]:
df.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


# **Basic Cleaning – Steps to Perform**

# Detecting Duplicate Values 

In [7]:
df.duplicated().sum()

146

# 1.Remove Duplicate Records
- Duplicate rows are repeated data points that add no value.
- Why to remove ?
- They bias the model by over-representing some samples.

- Removing duplicates prevents skewed learning.

In [8]:
df = df.drop_duplicates()
df.head(4)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63


In [9]:
df.shape

(53794, 10)

In [10]:
df["cut"].unique()

array(['Ideal', 'Premium', 'Good', 'Very Good', 'Fair'], dtype=object)

# **1.Detecting Missing Values(null values)**

In [11]:
df.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

# Outliers:
Outliers are data points that lie far away from the majority of values and can:
- Skew mean & variance
- Affect ML models (Linear Regression, KNN, SVM)
- Reduce model accuracy

# **3.Detecting Outliers**
- IQR method

In [12]:
num_cols = df.select_dtypes(include = ['int64','float64']).columns
cat_cols = df.select_dtypes(include = ['object']).columns

In [13]:
num_cols

Index(['carat', 'depth', 'table', 'price', 'x', 'y', 'z'], dtype='object')

In [14]:
cat_cols

Index(['cut', 'color', 'clarity'], dtype='object')


**1.IQR based**

In [15]:
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    print(col)
    print("IQR:",IQR)
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR
    print(f"Lower_limit:{lower_limit}")
    print(f"Upper_limit:{upper_limit}")
    print("MAX:",df[col].max())
    print("MIN:",df[col].min())

    outliers = df[(df[col] < lower_limit) | (df[col] > upper_limit)]
    print(f"{col} outlier count:" , outliers.shape[0])
    
    print("-"*40)

carat
IQR: 0.64
Lower_limit:-0.5599999999999999
Upper_limit:2.0
MAX: 5.01
MIN: 0.2
carat outlier count: 1873
----------------------------------------
depth
IQR: 1.5
Lower_limit:58.75
Upper_limit:64.75
MAX: 79.0
MIN: 43.0
depth outlier count: 2525
----------------------------------------
table
IQR: 3.0
Lower_limit:51.5
Upper_limit:63.5
MAX: 95.0
MIN: 43.0
table outlier count: 604
----------------------------------------
price
IQR: 4375.75
Lower_limit:-5612.625
Upper_limit:11890.375
MAX: 18823
MIN: 326
price outlier count: 3523
----------------------------------------
x
IQR: 1.83
Lower_limit:1.9649999999999999
Upper_limit:9.285
MAX: 10.74
MIN: 0.0
x outlier count: 31
----------------------------------------
y
IQR: 1.8200000000000003
Lower_limit:1.9899999999999993
Upper_limit:9.27
MAX: 58.9
MIN: 0.0
y outlier count: 28
----------------------------------------
z
IQR: 1.12
Lower_limit:1.23
Upper_limit:5.710000000000001
MAX: 31.8
MIN: 0.0
z outlier count: 48
---------------------------------

# Observation
- Diamonds data is **naturally skewed**
- High-price diamonds are rare but real
- Large carat diamonds are genuinely expensive
- So these are not errors, they are valid observations.

# Problem with Capping:
- Capping would Destroy real patterns , Harm price prediction , Reduce R² badly

- Although many features showed statistical outliers, they represent genuine high-value diamonds. Since the problem is price prediction, these extreme values are important. Therefore,
- **for carat** - use 99th percentile capping 
- **for Price** – Use Log Transform
- **for x, y, z** - remove impossible value(0's)

In [16]:
import numpy as np

upper_limit = df['carat'].quantile(0.99)
df['carat'] = np.where(df['carat'] > upper_limit, upper_limit, df['carat'])

In [17]:
df['price'] = np.log1p(df['price'])

# Removing 0 from x, y ,z
- A diamond: Must have length, width, height
- Cannot have dimension = 0 mm

- x = 0 or y = 0 or z = 0 → measurement error / data entry error

In [18]:
# Remove invalid rows (x, y, z should not be zero)
df = df[(df["x"] > 0) & (df["y"] > 0) & (df["z"] > 0)]

In [19]:
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,5.789960,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,5.789960,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,5.793014,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,5.814131,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,5.817111,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,7.922261,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,7.922261,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,7.922261,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,7.922261,6.15,6.12,3.74


#  selection of target variable and predictors

In [20]:
y = df["price"]
print(y.shape)

(53775,)


In [21]:
X = df.drop(["price"],axis = 1)
X

,carat,cut,color,clarity,depth,table,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,6.15,6.12,3.74


# spliting data into Train and Test data

In [22]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25,random_state = 42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(40331, 9)
(13444, 9)
(40331,)
(13444,)


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53775 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53775 non-null  float64
 1   cut      53775 non-null  object 
 2   color    53775 non-null  object 
 3   clarity  53775 non-null  object 
 4   depth    53775 non-null  float64
 5   table    53775 non-null  float64
 6   price    53775 non-null  float64
 7   x        53775 non-null  float64
 8   y        53775 non-null  float64
 9   z        53775 non-null  float64
dtypes: float64(7), object(3)
memory usage: 4.5+ MB


In [24]:

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
cat_cols = X_train.select_dtypes(include="object").columns
num_cols = X_train.select_dtypes(exclude="object").columns

preprocessor = ColumnTransformer(transformers=[("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
                                                ("num", StandardScaler(), num_cols)])


from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor

pipe = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("knn", KNeighborsRegressor())])


In [25]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11],
    "knn__weights": ["uniform", "distance"],
    "knn__p": [1, 2]
}
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=5,
    scoring="r2",
    verbose=1
)
grid.fit(X_train, y_train)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...Regressor())])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'knn__n_neighbors': [3, 5, ...], 'knn__p': [1, 2], 'knn__weights': ['uniform', 'distance']}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'r2'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 

In [26]:
print("Best Parameters:", grid.best_params_)
print("Best CV accuracy:", grid.best_score_)


Best Parameters: {'knn__n_neighbors': 7, 'knn__p': 1, 'knn__weights': 'distance'}
Best CV accuracy: 0.9835738181567797


In [27]:
final_model = grid.best_estimator_
# final trained model with best parameters

# Testing Performance

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

y_pred = final_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("MAE :", mae)
print("RMSE:", rmse)
print("R2  :", r2)


# Training Performance

In [27]:
y_train_pred = final_model.predict(X_train)

mae = mean_absolute_error(y_train, y_train_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2 = r2_score(y_train, y_train_pred)

print("MAE :", mae)
print("RMSE:", rmse)
print("R2  :", r2)


MAE : 0.00021730606580577834
RMSE: 0.004594737976942723
R2  : 0.999979503251284


In [58]:
import pickle

with open("diamond_knn_model.pkl", "wb") as f:
    pickle.dump(final_model, f)

print("Model saved successfully")


Model saved successfully
